In [1]:
%%sh

cd sample_img/

ls

bbc_art10_img1_ee0f70b0-1223-11f0-a820-05f89b05038b.jpg.webp
bbc_art10_img2_e8b985f0-1233-11f0-afc6-d1fc00c8c982.jpg.webp
bbc_art11_img1_33bbc210-11ae-11f0-9728-8ffe3122ba10.jpg.webp
bbc_art12_img1_bb690240-1181-11f0-ac9f-c37d6fd89579.jpg.webp
bbc_art13_img1_c6767c90-115d-11f0-ba12-8d27eb561761.jpg.webp
bbc_art13_img2_ef4dcf10-115d-11f0-ba12-8d27eb561761.png.webp
bbc_art13_img3_4783bfa0-115e-11f0-ba12-8d27eb561761.jpg.webp
bbc_art14_img1_c9468d90-0e1d-11f0-8331-61229d24cb73.jpg.webp
bbc_art14_img2_f3dc7570-0e1c-11f0-8331-61229d24cb73.jpg.webp
bbc_art14_img3_3854b000-108e-11f0-94b3-d3222284d093.jpg.webp
bbc_art14_img4_7db4b8e0-0e25-11f0-b851-29ea4ea8a4cd.jpg.webp
bbc_art14_img5_627683e0-0e1d-11f0-8331-61229d24cb73.jpg.webp
bbc_art14_img6_314dd430-0e1d-11f0-8331-61229d24cb73.jpg.webp
bbc_art2_img2_9208d7b0-1134-11f0-ac9f-c37d6fd89579.jpg.webp
bbc_art2_img3_8ec1d2e0-120c-11f0-ba12-8d27eb561761.jpg.webp
bbc_art2_img4_c5d5cc60-1134-11f0-ba12-8d27eb561761.jpg.webp
bbc_art2_img5_578af9f0-1135

In [4]:
import os
from PIL import Image
import torch
import torchvision.transforms as T

# ── CONFIGURE ───────────────────────────────────────────────────────────────
img_dir = "sample_img"     # Change to your image directory
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ── DEFINE TRANSFORM ─────────────────────────────────────────────────────────
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# ── ITERATE & TRANSFORM ──────────────────────────────────────────────────────
tensors = []
filenames = []

for fname in os.listdir(img_dir):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
        continue

    path = os.path.join(img_dir, fname)
    try:
        img = Image.open(path).convert("RGB")
    except Exception as e:
        print(f"Skipping {fname}: cannot open ({e})")
        continue

    x = transform(img)           # Tensor shape (3×224×224), dtype=torch.float
    x = x.to(device)             # optionally move to GPU
    tensors.append(x)
    filenames.append(fname)

# Stack into a single batch tensor if you like
batch = torch.stack(tensors, dim=0)  # shape (N, 3, 224, 224)

print(f"Processed {len(tensors)} images into a batch of shape {batch.shape}")


Processed 33 images into a batch of shape torch.Size([33, 3, 224, 224])


In [2]:

!pip install --upgrade transformers accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 835.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00


In [3]:
# Colab cell
from PIL import Image
import os, re, pandas as pd
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# Ensure we use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load model & processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model     = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
model.eval()

# Point this to your folder of downloaded images
img_dir = "/content/sample_img"  # or wherever you uploaded your images

results = []
for fname in os.listdir(img_dir):
    if not fname.lower().endswith((".jpg",".jpeg",".png",".webp")):
        continue
    path = os.path.join(img_dir, fname)
    try:
        img = Image.open(path).convert("RGB")
    except:
        print("Could not open", fname)
        continue

    # Generate a caption
    inputs = processor(images=img, return_tensors="pt").to(device)
    generated = model.generate(**inputs, max_new_tokens=30)
    caption = processor.decode(generated[0], skip_special_tokens=True)

    # Simple regex-based counts
    k = len(re.findall(r"\bkilled\b", caption, flags=re.I))
    d = len(re.findall(r"\bdied\b",   caption, flags=re.I))

    results.append({
        "file": fname,
        "caption": caption,
        "killed_mentions": k,
        "died_mentions": d
    })

df = pd.DataFrame(results)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

ModuleNotFoundError: No module named 'ace_tools'

In [ ]:
df

,file,caption,killed_mentions,died_mentions
0,bbc_art13_img2_ef4dcf10-115d-11f0-ba12-8d27eb5...,syria map showing the location of the syrian war,0,0
1,bbc_art10_img2_e8b985f0-1233-11f0-afc6-d1fc00c...,a large iceberg floating in the ocean,0,0
2,bbc_art13_img1_c6767c90-115d-11f0-ba12-8d27eb5...,a bomb is seen in the middle east of syria,0,0
3,bbc_art14_img2_f3dc7570-0e1c-11f0-8331-61229d2...,a man ' s hand with a tattoo on it,0,0
4,bbc_art2_img8_314643c0-1136-11f0-ac9f-c37d6fd8...,a young boy sits in front of a destroyed building,0,0
5,bbc_art14_img1_c9468d90-0e1d-11f0-8331-61229d2...,two women walk through the rubble of a destroy...,0,0
6,bbc_art4_img2_4059d510-1317-11f0-ba12-8d27eb56...,president trump signs the white house,0,0
7,bbc_art7_img1_19679e40-12bf-11f0-b234-07dc7691...,two women in red and blue jackets,0,0
8,bbc_art4_img1_aa98f530-1314-11f0-ac9f-c37d6fd8...,trump ' s presidential plan is a big deal,0,0
9,bbc_art6_img1_f934b120-12a1-11f0-ac9f-c37d6fd8...,a man in a suit and tie is surrounded by repor...,0,0
